# Phase 2, 3 and 4: Search, Augment and Generate the Answer
In this notebook there are several parts:
- Import libraries, load configuration variables and create clients
- Hybrid search with Semantic ranker
- Filter the chunks leaving the most relevant compared with the user's question
- Generate the answer for the query using the most relevante chunks as the context

### Import libraries, load configuration variables and create clients

In [ ]:
#%pip install python-dotenv
#%pip install openai
#%pip install tiktoken
#%pip install azure-search-documents

In [1]:
# Import libraries
import os
import sys
import json
import time
import pandas as pd
from dotenv import load_dotenv, find_dotenv
from openai import AzureOpenAI

sys.path.append(os.path.abspath('..'))
from common_utils import *

# Load Azure OpenAI and AI Search variables and create clients
openai_config, ai_search_config = load_config()


aoai_endpoint: https://openai-asc-swit-north.openai.azure.com/
aoai_deployment_name: gpt-4o
oai_embedding_model: ada
aoai_rerank_model: gpt-4o-mini
ai_search_index_name_regs: rag-index-regs
ai_search_index_name_docs: rag-index-docs


## Process Step by Step
1. Search in AI Search
2. Filter relevant chunks
3. Generate answer

### 1. Search in AI Search with hybrid (keyword and vector searches) with semantic ranker

In [2]:
# Generate the query for the question
question = "What is included in my Northwind Health Plus plan?"

# Hybrid search
results, num_results = semantic_hybrid_search(ai_search_config["ai_search_client_docs"],
                                              openai_config["openai_client"],
                                              openai_config["aoai_embedding_model"],
                                              question, 10)
print(f"num results: {num_results}")
print(f"num len(results): {len(results)}")
show_results(results, question)


num results: 144
num len(results): 10
Hybrid Search Results: [
  {
    "id": "141",
    "title": "Northwind_Health_Plus_Benefits_Details.md",
    "content": " beyond what is\nprovided by Workers' Compensation Insurance. These benefits may include disability\nbenefits, unemployment benefits, or Social Security benefits. It is important for employees\nto research these options in order to determine if they are eligible for any additional\nbenefits.\n\nWhen an employee is injured or becomes ill, they should contact the Workers'\nCompensation Insurance provider immediately. The provider will provide the employee\nwith information on the process and how to file a claim. The provider may also provide\nadditional resources to help the employee understand their rights and responsibilities.\n\nIt is important for employees to remember that Workers' Compensation Insurance is a\nbenefit that is provided by the employer. It is the employer's responsibility to ensure that\nemployees are aware of th

#### 2. Filter the retrieved chunks compared with the user's question

In [3]:
# Valid chunks for the user question
valid_chunks, num_chunks = get_filtered_chunks(openai_config["openai_client"],
                                               openai_config["aoai_rerank_model"],
                                               results,
                                               question)
print(f"num valid chunks: {num_chunks}")


num valid chunks: 3


#### 3. Generate the answer using the relevant chunks as context

In [4]:
# Generate answer:
answer = generate_answer(openai_config["openai_client"],
                         openai_config["aoai_deployment_name"],
                                       valid_chunks,
                                       question)
print(f"\n>> Answer:\n{answer}")



Calling Azure OpenAI model gpt-4o...

>> Answer:
Your Northwind Health Plus plan includes the following:

- **Copayment**: A fixed amount paid for a covered service at the time of service.
- **Deductible**: $2,000 per year, which is the amount you must pay out-of-pocket before the plan begins covering services.
- **Coinsurance**: 20% of the cost of a covered service after meeting the deductible.
- **Out-of-Pocket Maximum**: $4,000 per year, including deductible, coinsurance, and copayments.
- **In-Network Providers**: Lower copayments and coinsurance amounts.
- **Out-of-Network Providers**: Higher copayments and coinsurance amounts.
- **Preventive Care**: Covered at 100% with no copayment, deductible, or coinsurance.
- **Prescription Drugs**: Subject to a copayment, varying by drug type.
- **Mental Health and Substance Abuse Services**: Subject to a copayment and deductible.
- **Emergency Services**: Subject to a copayment and deductible, varying by network status.

Always verify if a

## End-to-end process

In [7]:
## End-to-end process:

import pandas as pd

# Read test data from Excel file
input_file = "../5_evaluation/ground_truth.xlsx"
df = pd.read_excel(input_file,)
data_dict = df.to_dict(orient='records')

question = ''

for i, line in enumerate(data_dict):

    question = line['QUESTION']

    print(f'[{i+1}] Question: {question}')
    # Hybrid search with Semantic ranker
    results, num_results = semantic_hybrid_search(ai_search_config["ai_search_client_docs"],
                                                openai_config["openai_client"],
                                                openai_config["aoai_embedding_model"],
                                                question, 50)
    print(f"num results: {num_results}")
    show_results(results, question)

    # Filter valid chunks for the user question
    valid_chunks, num_chunks = get_filtered_chunks(openai_config["openai_client"],
                                                openai_config["aoai_rerank_model"],
                                                results, question)

    # Generate answer:
    answer = generate_answer(openai_config["openai_client"],
                            openai_config["aoai_deployment_name"],
                            valid_chunks, question)
    print(f"\n>> Answer: {answer}")

[1] Question: What is included in my Northwind Health Plus plan?
num results: 144
Hybrid Search Results: [
  {
    "id": "141",
    "title": "Northwind_Health_Plus_Benefits_Details.md",
    "content": " beyond what is\nprovided by Workers' Compensation Insurance. These benefits may include disability\nbenefits, unemployment benefits, or Social Security benefits. It is important for employees\nto research these options in order to determine if they are eligible for any additional\nbenefits.\n\nWhen an employee is injured or becomes ill, they should contact the Workers'\nCompensation Insurance provider immediately. The provider will provide the employee\nwith information on the process and how to file a claim. The provider may also provide\nadditional resources to help the employee understand their rights and responsibilities.\n\nIt is important for employees to remember that Workers' Compensation Insurance is a\nbenefit that is provided by the employer. It is the employer's responsibili